In [ ]:
from pathlib import Path
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import cartopy.crs as ccrs
import numpy as np
import dmidc
import dmidc.harmonie
import dmidc.metobs
import dmidc.utils
import tempfile
from mllam_verification.plot import plot_single_metric_gridded_map
from mllam_verification.operations.statistics import rmse
from datetime import datetime

dmidc.__version__

In [ ]:
ANALYSIS_TIME = datetime(2022, 1, 1)

In [ ]:
version = "v0.5.0"
# path to copies of processed DANRA datasets on scale.dmi.dk:
fp_root = Path(f"/dmidata/projects/cloudphysics/danra/data/{version}")

ds_danra_sl = xr.open_zarr(fp_root / "single_levels.zarr")
ds_danra_sl.attrs['suite_name'] = "danra"

In [ ]:
with tempfile.TemporaryDirectory() as tempdir:
    ds = dmidc.harmonie.load(
        suite_name="DANRA",
        analysis_time=ANALYSIS_TIME.strftime("%Y-%m-%d"),
        data_kind="FORECAST",
        forecast_duration="PT3H",
        short_name=["t"],
        level_type="heightAboveGround",
        level=2,
        temp_filepath=tempdir,
    )


In [ ]:
ds

In [ ]:
da1_t2m = ds_danra_sl["t2m"].sel(time=slice("2022-01-01", "2022-01-05"))
da2_t2m = ds_danra_sl["t2m"].sel(time=slice("2022-03-01", "2022-03-05"))
# shift time of da2_t2m to match that of da1_t2m
da2_t2m = da2_t2m.assign_coords(time=da1_t2m.time)
# bbox = [7.5, 15.5, 54.5, 58.5]

fig, ax = plt.subplots(figsize=(8, 10), subplot_kw={"projection": ccrs.AlbersEqualArea(central_longitude=11.0, central_latitude=55.0)})

ax = plot_single_metric_gridded_map(da1_t2m, da2_t2m, time_selection=datetime(2022, 1, 1), axes=ax, xarray_plot_kwargs={"transform": ccrs.PlateCarree()})
ax.coastlines(linewidth=0.5, color="black")
ax.gridlines(draw_labels=["top", "left"], color="gray", alpha=0.5, linestyle="--")
# ax.set_extent(bbox, crs=ccrs.PlateCarree())

In [ ]:
ds_danra_sl["t2m"]["x"].values